In [2]:
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix, save_npz




def gen_Smatrix(input_file, beta=1.0, alpha=0.5):
    #  READ EDGES AND EXTRACT ALL UNIQUE NODES
    connections = []
    all_nodes = set()
    with open(input_file, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue
            src, tgt = map(int, line.strip().split())
            connections.append((src, tgt))
            all_nodes.add(src)
            all_nodes.add(tgt)
    
    node_list = list(all_nodes)
    id_to_index = {node_id: idx for idx, node_id in enumerate(node_list)}
    total_nodes = len(node_list)
    
    # CONSTRUCT ADJACENCY MATRIX (SPARSE FORMAT)
    adjacency = lil_matrix((total_nodes, total_nodes), dtype=np.float32)
    for src, tgt in connections:
        i = id_to_index[src]
        j = id_to_index[tgt]
        adjacency[i, j] = 1
        adjacency[j, i] = 1  # UNDIRECTED EDGE: ADD BOTH DIRECTIONS
    
    adjacency = adjacency.tocsr()
    
    # COMPUTE A^2 → PATHS OF LENGTH 2
    adjacency_squared = adjacency.dot(adjacency)
    
    # CALCULATE SENSITIVITY MATRIX USING FORMULA
    sensitivity = beta * adjacency + beta * alpha * adjacency_squared
    
    return sensitivity, id_to_index


'''
(0, 0)        4.0      ← STRONG SELF-AFFINITY (BASE + INDIRECT PATHS)
(0, 87048)    2.0      ← STRONG LINK BETWEEN PRODUCT 0 AND 87048
(0, 249812)   1.5      ← MODERATE INFLUENCE VIA PATHS OF LENGTH 1 OR 2
(0, 168876)   1.5
(0, 173665)   1.5
(0, 246086)   1.5
(0, 11449)    1.0      ← WEAKER TIES THROUGH SPARSE PATHWAYS
'''

#  RUN AND SAVE SENSITIVITY MATRIX
if __name__ == "__main__":
    matrix, node_map = gen_Smatrix('Database.txt')
    save_npz('sensitivity_matrix.npz', matrix)
    print(matrix)
    print("Sensitivity matrix saved as 'sensitivity_matrix.npz'")


  (0, 0)	4.0
  (0, 29452)	0.5
  (0, 61822)	0.5
  (0, 97093)	0.5
  (0, 249812)	1.5
  (0, 5187)	0.5
  (0, 11449)	1.0
  (0, 26814)	0.5
  (0, 46673)	0.5
  (0, 68446)	0.5
  (0, 87048)	2.0
  (0, 168876)	1.5
  (0, 173665)	1.5
  (0, 246086)	1.5
  (0, 266878)	0.5
  (0, 289713)	0.5
  (0, 301174)	1.0
  (0, 309355)	1.0
  (0, 21846)	0.5
  (0, 31802)	1.0
  (0, 91490)	0.5
  (0, 94663)	1.5
  (0, 102482)	0.5
  (0, 124741)	0.5
  (0, 134926)	1.0
  :	:
  (334862, 57258)	0.5
  (334862, 59896)	0.5
  (334862, 71899)	0.5
  (334862, 82171)	0.5
  (334862, 107829)	0.5
  (334862, 132367)	0.5
  (334862, 156754)	0.5
  (334862, 175944)	0.5
  (334862, 187159)	0.5
  (334862, 210119)	0.5
  (334862, 222367)	0.5
  (334862, 233726)	0.5
  (334862, 237360)	0.5
  (334862, 243508)	0.5
  (334862, 251195)	0.5
  (334862, 262084)	0.5
  (334862, 273845)	0.5
  (334862, 293249)	0.5
  (334862, 303011)	0.5
  (334862, 314099)	0.5
  (334862, 325240)	0.5
  (334862, 326794)	0.5
  (334862, 331955)	0.5
  (334862, 212820)	1.0
  (334862, 1173